In [17]:
# Importar librerías para manipulación y análisis de datos
# -----------------------------------------------------------------------
import pandas as pd

# Importar librerías para automatización de navegadores web
# -----------------------------------------------------------------------
from time import sleep #Para el sleep
from tqdm import tqdm #Para la barra de porcentaje
from selenium import webdriver #Para llamar a la web
from webdriver_manager.chrome import ChromeDriverManager #Para llamar a la web
from selenium.webdriver.common.keys import Keys 
#from selenium.webdriver.chrome.options import Options #Opciones
#from selenium.webdriver.support.ui import Select #Desplegables

# Importar librería para hacer solicitudes HTTP
# -----------------------------------------------------------------------
import requests
import re

# Importar librería para suprimir advertencias
# -----------------------------------------------------------------------
import warnings #Para ignorar los mensajes de warning que nos salen tipo que esta un bot controlando la pagina
warnings.filterwarnings('ignore')

import os

from dotenv import load_dotenv
load_dotenv()

from bs4 import BeautifulSoup


In [18]:
url = f"https://turismoactiva.com/rutas-de-montana-sierra-de-gredos/"
#Llamo a la url
respuesta = requests.get(url)
print("La respuesta de la petición es:", respuesta.status_code)
respuesta.content
sopa = BeautifulSoup(respuesta.content, 'html.parser')
lista_detalles = sopa.find_all("ul")
#Al movernos por la lista encontramos todos los elementos pedidos salvo la descripción
#En la 0 hay datos sobre los materiales obligatorios
lista_detalles[6].text.split("\n")
#Elemento 0 de nuestro split " " y últimos no nos interessan. Cogemos solo del 1 al 6 (incluidos)
    

La respuesta de la petición es: 200


['',
 'Donde se realizan:\xa0Sierra de Bejar, salida desde la Plataforma del Travieso, sector occidental',
 'Tipo de ruta: Ida y vuelta. (Posibilidad de circular por la Dehesa)',
 'Duración:\xa07 horas.',
 'km: 12',
 'Dificultad técnica:\xa0Media. (Dependerá de la estación del año).',
 'Esfuerzo: medio.',
 'Personas mínimas:\xa04 personas.',
 'Puntos de interés:\xa0Nacimiento del Rio Cuerpo de Hombre, vistas del Circo. Impresionantes vistas de los diferentes sectores de Gredos y la meseta.',
 'Época: Anual',
 'Precio: 30€/ persona actividad sin nieve, 40€/persona actividad invernal (preguntar precio para grupos 10 o superior)',
 '']

In [19]:
lista_rutas = sopa.find_all("h4", {"class":"elementor-heading-title elementor-size-default"})
lista_rutas

[<h4 class="elementor-heading-title elementor-size-default">Pico Morenzon</h4>,
 <h4 class="elementor-heading-title elementor-size-default">Pico de la Mira</h4>,
 <h4 class="elementor-heading-title elementor-size-default">Pico Almanzor</h4>,
 <h4 class="elementor-heading-title elementor-size-default">Laguna Grande</h4>,
 <h4 class="elementor-heading-title elementor-size-default">Cinco Lagunas</h4>,
 <h4 class="elementor-heading-title elementor-size-default">Circo de  Hoya Moros</h4>,
 <h4 class="elementor-heading-title elementor-size-default">Canchal Negro</h4>,
 <h4 class="elementor-heading-title elementor-size-default">El Calvitero</h4>,
 <h4 class="elementor-heading-title elementor-size-default">Canchal de la Ceja, Torreon, Calvitero</h4>,
 <h4 class="elementor-heading-title elementor-size-default">Laguna de Barco</h4>,
 <h4 class="elementor-heading-title elementor-size-default">Tres Lagunas, La Nava, Barco y Caballeros</h4>,
 <h4 class="elementor-heading-title elementor-size-defaul

In [33]:
url = f"https://turismoactiva.com/rutas-de-montana-sierra-de-gredos/"
#Llamo a la url
respuesta = requests.get(url)
print("La respuesta de la petición es:", respuesta.status_code)
respuesta.content
sopa = BeautifulSoup(respuesta.content, 'html.parser')
lista_detalles = sopa.find_all("ul")
#Al movernos por la lista encontramos todos los elementos pedidos salvo la descripción
#En la 0 hay datos sobre los materiales obligatorios
lista_detalles[6].text.split("\n")
#Elemento 0 de nuestro split " " y últimos no nos interessan. Cogemos solo del 1 al 6 (incluidos)
    
diccionario = {"nombre":[],
               "donde":[],
               "tipo":[],
               "duración":[],
               "km":[],
               "dificultad":[],
               "esfuerzo":[],
               "descripción":[]}

#Metemos los nombres de las rutas en el diccionario
for i in range(1,14):
    lista_rutas = sopa.find_all("h4", {"class":"elementor-heading-title elementor-size-default"})
    diccionario["nombre"].append(lista_rutas[i].text)
  
#Nos movemos por los detalles a sacar
for elemento in lista_detalles[1:14]:
    #Elemento 0 de nuestro split " " y últimos no nos interessan. Cogemos solo del 1 al 6 (incluidos)
    for detalle in range(1,7):
        cada_elemento = elemento.text.split("\n")[detalle].split(": ")
        
        if detalle == 1:
            diccionario["donde"].append(cada_elemento)
        elif detalle == 2:
            diccionario["tipo"].append(cada_elemento)
        elif detalle == 3:
            diccionario["duración"].append(cada_elemento)
        elif detalle == 4:
            diccionario["km"].append(cada_elemento)
        elif detalle == 5:
            diccionario["dificultad"].append(cada_elemento)            
        elif detalle == 6:
            diccionario["esfuerzo"].append(cada_elemento)            


#Pico Morenzon
descripcion = sopa.find_all("p", {"style":"font-size: 16.875px; font-style: normal;"})
diccionario["descripción"].append(descripcion[1].text)

#Buscando el resto --> Iterando de 3 en 3 --> Lo resolvimos en clase con [::3]
descripcion = sopa.find_all("div", {"class":"elementor-widget-container"})
#Pico de la Mira
diccionario["descripción"].append(descripcion[19].text.split("Descripción:\n")[1])

#Pico Almanzor
diccionario["descripción"].append(descripcion[22].text.split("Descripción:\n")[1])

#Laguna Grande
diccionario["descripción"].append(descripcion[25].text.split("Descripción:\n")[1])

#Cinco Lagunas
diccionario["descripción"].append(descripcion[28].text.split("Descripción:\n")[1])

#Circo de Hoya Moros
diccionario["descripción"].append(descripcion[31].text.split("Descripción:\n")[1])

#Canchal Negro
diccionario["descripción"].append(descripcion[34].text.split("Descripción:\n")[1])

#El Calvitero
diccionario["descripción"].append(descripcion[37].text.split("Descripción:\n")[1])

#Canchal de la Ceja, Torreon, Calvitero
diccionario["descripción"].append(descripcion[40].text.split("Descripción:\n")[1])

#Laguna de Barco
diccionario["descripción"].append(descripcion[43].text.split("Descripción:\n")[1])

#Tres Lagunas, La Nava, Barco y Caballeros
diccionario["descripción"].append(descripcion[46].text.split("Descripción:\n")[1])

#Laguna de los Caballeros
diccionario["descripción"].append(descripcion[49].text.split("Descripción:\n")[1])

#Laguna de la Nava
diccionario["descripción"].append(descripcion[52].text.split("Descripción:\n")[1])


diccionario

La respuesta de la petición es: 200


{'nombre': ['Pico de la Mira',
  'Pico Almanzor',
  'Laguna Grande',
  'Cinco Lagunas',
  'Circo de  Hoya Moros',
  'Canchal Negro',
  'El Calvitero',
  'Canchal de la Ceja, Torreon, Calvitero',
  'Laguna de Barco',
  'Tres Lagunas, La Nava, Barco y Caballeros',
  'Laguna de los Caballeros',
  'Laguna de la Nava',
  'Nº Registro TA 37/23'],
 'donde': [['Donde se realizan:\xa0Sierra de Gredos, salida desde la Plataforma de Gredos. Sector Central'],
  ['Donde se realizan:\xa0Sierra de Gredos, salida desde la Plataforma de Gredos. Sector Central'],
  ['Donde se realizan:\xa0Sierra de Gredos, salida desde la Plataforma de Gredos. Sector Central'],
  ['Donde se realizan:\xa0Sierra de Gredos, Plataforma de Gredos, sector central.'],
  ['Donde se realizan:\xa0Sierra de Gredos, salida desde Navalperal del Tormes. Sector central.'],
  ['Donde se realizan:\xa0Sierra de Bejar, salida desde la Plataforma del Travieso, sector occidental'],
  ['Localización', 'Sierra de Bejar, Gredos Oeste\xa0(Salam

In [29]:
print(len(diccionario["nombre"]),len(diccionario["donde"]),len(diccionario["tipo"]),len(diccionario["duración"]),len(diccionario["km"]),len(diccionario["dificultad"]),len(diccionario["esfuerzo"]),len(diccionario["km"]))

13 13 13 13 13 13 13 13


In [21]:
df_rutas = pd.DataFrame(diccionario)

df_rutas

,nombre,donde,tipo,duración,km,dificultad,esfuerzo,descripción
0,Pico de la Mira,"[Donde se realizan: Sierra de Gredos, salida d...",[Tipo de ruta: circular.],[Duración: 4 horas.],"[Km, 10]",[Dificultad Técnica: baja. (Dependerá de la es...,[Esfuerzo: bajo – medio],"Aparte de las rutas que aquí especificamos, po..."
1,Pico Almanzor,"[Donde se realizan: Sierra de Gredos, salida d...","[Tipo de ruta, Ida y vuelta.]",[Duración: 7 horas.],"[km, 16]",[Dificultad técnica: Media. (Dependerá de la e...,"[Esfuerzo, Medio]","Una ruta perfecta para iniciarse en Gredos, si..."
2,Laguna Grande,"[Donde se realizan: Sierra de Gredos, salida d...","[Tipo de ruta, Ida y vuelta.]",[Duración: 14 horas.],"[km, 22]",[Dificultad técnica: Alta. (Dependerá de la es...,"[Esfuerzo, Alto]",Subir al pico Almanzor es una actividad impres...
3,Cinco Lagunas,"[Donde se realizan: Sierra de Gredos, Platafor...","[Tipo de ruta, Ida y vuelta.]",[Duración: 5 horas.],"[Km, 12]",[Dificultad técnica: baja. (Dependerá de la es...,"[Esfuerzo, Bajo – medio.]","Ruta conocida por todos, La Laguna Grande de G..."
4,Circo de Hoya Moros,"[Donde se realizan: Sierra de Gredos, salida d...","[Tipo de ruta, Ida y vuelta.]",[Duración: 9 horas.],"[Km, 25]",[Dificultad técnica: media. (Dependerá de la e...,"[Esfuerzo, alto]","Saliendo desde Navalperal del Tormes, nos diri..."
5,Canchal Negro,"[Donde se realizan: Sierra de Bejar, salida de...","[Tipo de ruta, Ida y vuelta. (Posibilidad de c...",[Duración: 7 horas.],"[km, 12]",[Dificultad técnica: Media. (Dependerá de la e...,"[Esfuerzo, medio.]",Una de las rutas más características de la Sie...
6,El Calvitero,"[Localización, Sierra de Bejar, Gredos Oeste (...","[Tiempo, 3,5 horas]","[Dificultad técnica, baja]","[Esfuerzo, bajo]","[Km, 8 km.]","[Punto más alto, 2355 metros]","Se trata de una ruta facil, bonita y apta para..."
7,"Canchal de la Ceja, Torreon, Calvitero","[Localización, Sierra de Bejar, Gredos Oeste (...","[Tiempo, 4,5 horas]","[Dificultad técnica, baja – media]","[Esfuerzo, Medio]","[Km, 14 km.]","[Punto más alto, 2400 metros]","Un clásico de la Sierra de Béjar, subir al Cal..."
8,Laguna de Barco,"[Donde se realizan: Sierra de Béjar, salida de...","[Tipo de ruta, circular.]","[Duración, 8 horas.]","[km, 16]",[Dificultad técnica: medio. (Dependerá de la e...,"[Esfuerzo, Alto]","El Calvitero, Canchal de La Ceja y Torreón, la..."
9,"Tres Lagunas, La Nava, Barco y Caballeros","[Donde se realizan: Sierra de Gredos, aparcami...","[Tipo de ruta, Ida y vuelta.]",[Duración: 7 horas.],"[Km, 21]",[Dificultad técnica: baja. (Dependerá de la es...,"[Esfuerzo, medio.]","No es una ruta muy transitada, al contrario qu..."


In [17]:
url_aemet = "https://opendata.aemet.es/opendata/api/prediccion/especifica/montaña/pasada/area/gre1"
#Llamamos a la api
respuesta = requests.get(url_aemet)

#Comprobamos si todo correcto
respuesta.status_code


200

In [18]:
json_resp = respuesta.json()
#json_resp

JSONDecodeError: Expecting value: line 1 column 1 (char 0)